## Assignment 3 - Competitive Assignment

### Preceding Step - import modules (packages)
Shay Mualem 200332435 <br/>

In [24]:
#Shay Mualem 200332435

In [25]:
%%capture
# imports for reading and writing (input & output) files:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import SGDClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn import metrics

from sklearn.pipeline import Pipeline

import os

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import FunctionTransformer

import string
import re


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [26]:
# Split into two data frames, one for metrics, the other for grouping
def load_dataset(file_name, category):
    data = pd.read_excel(file_name,  'corpus', index_col=None, na_values=['NA'])
    return (data.loc[:, data.columns != category].squeeze() , data[category].squeeze())

In [27]:
train_filename = '.' + os.sep + 'input' + os.sep + 'annotated_corpus_for_train.xlsx'
test_filename  = '.' + os.sep + 'input' + os.sep + 'corpus_for_test.xlsx'

#load the xlsx files
df_test  = pd.read_excel(test_filename,  'corpus', index_col = None, na_values=['NA'])
df_train = pd.read_excel(train_filename, 'corpus', index_col = None, na_values=['NA'])

#split df using load_dataset fun
df_train_story, df_train_gender = load_dataset(train_filename,'gender')

#conver to binary, not use to see
# df_train_gender = df_train_gender.replace("f", "1")
# df_train_gender = df_train_gender.replace("m", "0")

In [28]:
# Create manually tokenizer

def my_tokenizer(text):
    # create a space between special characters
    text = re.sub("(\\W)"," \\1 ",text)
    # split based on whitespace
    return re.split("\\s+" ,text)    

# my FunctionTransformer

In [29]:
#word thar have 'ani' and  with 'hii' can have 'maod' or 'mmas'

def num_of_word_ani_end_hay(text):
    text_with_hay = (re.findall(r"(((אני|הייתי)( )(ממש|מאוד)?( )?[א-ת]+[ה][\s]))",text))
     #if have make it strong vec and need to be more then 8 letr

    return len(text_with_hay)

In [30]:
#find 'ben-zogy word'

def num_of_zogi(text):
    text_with_hay = (re.findall(r"((בן)( )(זוג))",text))
        
    return len(text_with_hay)

In [31]:
#word thar have 'ani' and  with 'taf' can have 'maod' or 'mmas'

def num_of_word_ani_end_tof_endmod_mmas(text):
    text_with_taf = (re.findall(r"(((אני|הייתי)( )(ממש|מאוד)?( )?[א-ת]+[ת][\s]))",text))
    
    return len(text_with_taf)

In [32]:
#word thar and with 'it'

def num_of_end_et(text):
    
    return len(re.findall("([א-ת]+(ית)[ ])", text)) 

In [33]:
# def avg_of_word_len(storys):
    
#     return (sum(len(word) for word in storys) / len(storys.split()))

In [34]:
# def number_of_new_lain(storys):
#     return len(re.findall("\n", storys))

In [35]:
# def num_of_numbers(text):
#     print(len(re.findall("\d+", text)))
#     return len(re.findall("\d+", text))

In [36]:
# #word thar and with 'vit'

# def num_of_end_v_et(text):
    
#     return len(re.findall("([א-ת]+(ות)[ ])", text)) 

# Pipeline

In [37]:
#using Pipeline: TfidfVectorizer > FunctionTransformer > SGDClassifier
# i got this number min_df = 3 , max_df = 0.9 by using alot of trying
    
classifier = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', TfidfVectorizer(min_df = 3 , max_df = 0.9 ,tokenizer = my_tokenizer)),
#             ('tfidf', TfidfTransformer()),
        ])),
        
#      ('try', Pipeline([
         ('ani_end_tof', FunctionTransformer(lambda x: np.array([num_of_word_ani_end_tof_endmod_mmas(t) for t in x]).reshape(-1, 1), validate=False)),
         ('num_of_bnZogi', FunctionTransformer(lambda x: np.array([num_of_zogi(t) for t in x]).reshape(-1, 1), validate=False)),
         ('num_of_word_ani_end_hay', FunctionTransformer(lambda x: np.array([num_of_word_ani_end_hay(t) for t in x]).reshape(-1, 1), validate=False)),
         ('new_lain', FunctionTransformer(lambda x: np.array([num_of_end_et(t) for t in x]).reshape(-1, 1), validate=False)),
#          ('num_of_numbers', FunctionTransformer(lambda x: np.array([num_of_end_v_et(t) for t in x]).reshape(-1, 1), validate=False)),
#          ('num_of_end_et', FunctionTransformer(lambda x: np.array([avg_of_word_len(t) for t in x]).reshape(-1, 1), validate=False)),
#          ]))
    ])),
    ('clf-svm', SGDClassifier(loss='hinge', penalty='l2' ,alpha=1e-3, tol=None))])

classifier =  classifier.fit(df_train_story, df_train_gender)

# classifier.named_steps["features"].get_feature_names()
# # pd.DataFrame(classifier.named_steps["vect"].get_feature_names())

### Computing cross-validated metrics


In [38]:
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

df_train, Y1 = load_dataset(train_filename,'gender')

#using  cross val with 6 fold cross validation using f1
scores = cross_val_score(classifier, df_train,Y1 , cv = 10, scoring = 'f1_macro')

print('max: ', scores.max())
print('mean/avg: ', scores.mean()) # < what we need
print('min: ', scores.min())

# max:  0.8937799043062201
# mean/avg:  0.8050399028861932
# min:  0.6210526315789474

max:  0.8844919786096257
mean/avg:  0.8009793727730316
min:  0.6210526315789474


### Your implementation:
Write your code solution in the following code-cells

In [39]:
# The stories from the test
df_test['story'].head()

0    כחלק ממסגרת ההתנדבות שלי במגלה אני הולך לפעמיי...
1    לפני שנה החלטתי שאני רוצה להיות טייס, התחלתי ל...
2    בתקופת הקורונה של תחילת החיסונים נגד קורונה, א...
3    כפי שכולם מכירים או שמעו מחברים עולם הדייטים ה...
4    אחת החוויות שהכי זכורות לי, זו החוויה בפרו בטי...
Name: story, dtype: object

In [40]:
#this is my predicten
my_test_predicted = classifier.predict(df_test['story'])
my_test_predicted

array(['m', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm',
       'm', 'm', 'f', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm',
       'm', 'f', 'm', 'm', 'm', 'f', 'm', 'f', 'm', 'm', 'm', 'f', 'm',
       'f', 'm', 'm', 'm', 'm', 'm', 'm', 'f', 'm', 'm', 'm', 'f', 'f',
       'm', 'm', 'm', 'f', 'f', 'f', 'm', 'm', 'm', 'm', 'm', 'm', 'f',
       'f', 'm', 'f', 'f', 'm', 'm', 'f', 'm', 'm', 'm', 'm', 'm', 'm',
       'm', 'm', 'f', 'm', 'm', 'm', 'm', 'f', 'm', 'm', 'm', 'm', 'm',
       'm', 'f', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm',
       'm', 'f', 'm', 'm', 'm', 'm', 'f', 'm', 'm', 'm', 'm', 'm', 'm',
       'm', 'f', 'm', 'm', 'f', 'm', 'f', 'm', 'f', 'm', 'm', 'm', 'm',
       'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'f',
       'm', 'm', 'm', 'f', 'f', 'm', 'f', 'm', 'm', 'm', 'm', 'm', 'f'],
      dtype='<U1')

In [41]:
# df_predicted = pd.DataFrame(df_test['test_example_id'], ndarray_predicted, columns = ['test_example_id','predicted_category'])
df_predicted = pd.DataFrame([df_test['test_example_id'] , my_test_predicted]).T
df_predicted.columns=['test_example_id','predicted_category']


In [42]:
# show the df_predicted id and predicted_category
df_predicted

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m
...,...,...
151,151,m
152,152,m
153,153,m
154,154,m


### Save output to csv
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [43]:
df_predicted.to_csv('classification_results.csv',index=False)

In [44]:
# ================